In [1]:
import xgboost as xgb
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

/home/rohit_mattah/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()

In [3]:
clf2 = RandomForestClassifier(n_estimators=2000)
clf3 = GradientBoostingClassifier(learning_rate=0.1, loss='deviance', max_depth=5,n_estimators=2000,subsample=.5)
clf6 = KNeighborsClassifier(n_neighbors=8,weights='distance',n_jobs=-1)
clf8 = KNeighborsClassifier(n_neighbors=2,weights='distance',n_jobs=-1)
clf9 = KNeighborsClassifier(n_neighbors=4,weights='distance',n_jobs=-1)
clf10 = KNeighborsClassifier(n_neighbors=8,weights='distance',n_jobs=-1)

In [4]:
gb = xgb.XGBClassifier(learning_rate =0.08, n_estimators=100, 
                       max_depth=15, min_child_weight=1,
                       gamma=2,subsample=0.4,
                       colsample_bytree=0.55,objective= 'multi:softprob',
                       nthread=4,scale_pos_weight=1,seed=27)

sclf = StackingCVClassifier(classifiers=[clf2, clf3, clf6, clf8, clf9,clf10], meta_classifier=gb)


In [5]:
train=pd.io.parsers.read_csv("train_submissions.csv")
train1 = train.drop('attempts_range', axis=1)
train1['user_id'] = number.fit_transform(train1['user_id'].astype('str'))
train1['problem_id'] = number.fit_transform(train1['problem_id'].astype('str'))
x=train1.values

In [6]:
y=train['attempts_range'].ravel()

In [20]:
test = pd.io.parsers.read_csv("test_submissions.csv")
test1 = test.drop('ID', axis=1)
test1['user_id'] = number.fit_transform(test1['user_id'].astype('str'))
test1['problem_id'] = number.fit_transform(test1['problem_id'].astype('str'))
t1=test1.values

In [10]:
st = model_selection.cross_val_score(gb, x, y, 
                                              cv=3, scoring='accuracy',n_jobs = -1) #Stacking model CV calculation
st.mean()

0.55001128471067562

In [11]:
model = gb.fit(x,y)

In [21]:
predicts = model.predict(t1)

In [23]:
i1 = pd.DataFrame({ 'attempts_range': predicts })
i3 = test['ID']
i2 = pd.DataFrame({ 'ID': i3 })
enigma = pd.concat([i2, i1], axis=1)
enigma.head()

,ID,attempts_range
0,user_856_prob_5822,1
1,user_2642_prob_2334,1
2,user_2557_prob_2920,1
3,user_1572_prob_4598,1
4,user_295_prob_6139,1


In [24]:
enigma.to_csv("enigma.csv", index=False)